# Neural network classifier tutorial

In this tutorial we will train a feedforward / MLP network classifier on the SVHN dataset.

In [ ]:
import tensorflow as tf
import numpy as np

This notebook assumes that the SVHN data has been downloaded and preprocessed ready for network training. This work is covered in Assignment 4; ensure that you have done this before beginning this notebook.

## Import the data

In [ ]:
# Add the paths below to the saved .npy files for training and test inputs and labels
training_set_path = '../../assignments/Day 4/training_set_gs.npy'
training_labels_path = '../../assignments/Day 4/training_labels_one_hot.npy'
test_set_path = '../../assignments/Day 4/test_set_gs.npy'
test_labels_path = '../../assignments/Day 4/test_labels_one_hot.npy'

training_set_gs = np.load(training_set_path)
training_labels = np.load(training_labels_path)
test_set_gs = np.load(test_set_path)
test_labels = np.load(test_labels_path)

`train` and `test` are dictionaries with keys `'X'` and `'y'`. The values are numpy arrays.

In [ ]:
print(training_set_gs.shape)
print(training_labels.shape)

In [ ]:
print(test_set_gs.shape)
print(test_labels.shape)

In [ ]:
n_train = training_set_gs.shape[0]
n_test = test_set_gs.shape[0]

### Inspect the data

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
example = np.random.choice(np.arange(n_train))

image = np.squeeze(training_set_gs[example])
label = np.where(training_labels[example] == 1)[0][0]

plt.imshow(image, cmap='gray')
plt.show()

print("Digit: {}".format(label))

### Flatten the inputs to feed into an MLP

In [ ]:
training_set_flat = training_set_gs.reshape((n_train, -1))
test_set_flat = test_set_gs.reshape((n_test, -1))

print(training_set_flat.shape)
print(test_set_flat.shape)

## Build the network

In [ ]:
class SVHN_MLP:
    def __init__(self, wd_factor, learning_rate):
        self.wd_factor = wd_factor
        self.learning_rate = learning_rate
        self.train_pointer = 0
        self.test_pointer = 0
        
        self.sess = tf.Session()
        
        self.input = tf.placeholder(dtype=tf.float32, shape=[None, 1024], name='input')
        self.ground_truth = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='ground_truth')
        print(self.input)
        
        self._build_graph()
        
    def _build_graph(self):
        weights = []  # for weight decay
        
        with tf.variable_scope('layers'):
            h = tf.layers.dense(self.input, 512, kernel_initializer=tf.glorot_uniform_initializer(), 
                                activation=tf.tanh, name='1')
            print(h)
            h = tf.layers.dense(h, 256, kernel_initializer=tf.glorot_uniform_initializer(), 
                                activation=tf.tanh, name='2')
            print(h)
            h = tf.layers.dense(h, 64, kernel_initializer=tf.glorot_uniform_initializer(), 
                                activation=tf.tanh, name='3')
            print(h)
            self.logits = tf.layers.dense(h, 10, kernel_initializer=tf.glorot_uniform_initializer(), 
                                          activation=tf.identity, name='4')
            print(self.logits)
            self.prediction = tf.nn.softmax(self.logits, name='softmax_prediction')
            
        with tf.name_scope('loss'):
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, 
                                                                                  labels=self.ground_truth))
            self.loss += self.weight_decay()
            
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.train_op = self.optimizer.minimize(self.loss)
            
    def weight_decay(self):
        loss = 0
        for v in tf.global_variables():
            if 'Adam' in v.name:
                continue
            elif 'kernel' in v.name:
                loss += self.wd_factor * tf.nn.l2_loss(v)
        print(loss)
        return loss
    
    def train_minibatch(self, samples, labels, batch_size):
        if self.train_pointer + batch_size <= samples.shape[0]:
            samples_minibatch = samples[self.train_pointer: self.train_pointer + batch_size]
            labels_minibatch = labels[self.train_pointer: self.train_pointer + batch_size]
            self.train_pointer += batch_size
        else:
            samples_minibatch = samples[self.train_pointer:]
            labels_minibatch = labels[self.train_pointer: self.train_pointer + batch_size]
            self.train_pointer = 0
        return samples_minibatch, labels_minibatch

    def train(self, train_samples, train_labels, train_batch_size, iteration_steps):
        self.sess.run(tf.global_variables_initializer())

        print('Start Training')
        losses = []
        for i in range(iteration_steps):
            samples, labels = self.train_minibatch(train_samples, train_labels, train_batch_size)
            feed_dict = {self.input: samples, self.ground_truth: labels}
            _, loss = self.sess.run([self.train_op, self.loss], feed_dict=feed_dict)
            if i % 50 == 0:
                print("Minibatch loss at step {}: {}".format(i, loss))
                losses.append([i, loss])
        return losses
                    
    def test_minibatch(self, samples, labels, batch_size):
        if self.test_pointer + batch_size <= samples.shape[0]:
            samples_minibatch = samples[self.test_pointer: self.test_pointer + batch_size]
            labels_minibatch = labels[self.test_pointer: self.test_pointer + batch_size]
            self.test_pointer += batch_size
            end_of_epoch = False
        else:
            samples_minibatch = samples[self.test_pointer:]
            labels_minibatch = labels[self.test_pointer: self.test_pointer + batch_size]
            self.test_pointer = 0
            end_of_epoch = True
        return samples_minibatch, labels_minibatch, end_of_epoch
            
    def test(self, test_samples, test_labels, test_batch_size):
        end_of_epoch = False
        losses = []
        while not end_of_epoch:
            samples, labels, end_of_epoch = self.test_minibatch(test_samples, test_labels, test_batch_size)
            feed_dict = {self.input: samples, self.ground_truth: labels}
            losses.append(self.sess.run(self.loss, feed_dict=feed_dict))  
        print("Average test loss: {}".format(np.mean(losses)))

In [ ]:
WD_FACTOR = 0.0001
LEARNING_RATE = 0.001
model = SVHN_MLP(WD_FACTOR, LEARNING_RATE)

In [ ]:
tf.global_variables()

### Train the network

In [ ]:
TRAIN_BATCH_SIZE = 128
ITERATIONS = 10000

import time
start_time = time.time()

losses = model.train(training_set_flat, training_labels, TRAIN_BATCH_SIZE, ITERATIONS)

end_time = time.time()
print("Training time: {}s".format(end_time - start_time))

In [ ]:
losses = np.array(losses)
print(losses.shape)

In [ ]:
plt.style.use('seaborn')
iterations = losses[:, 0]
train_loss = losses[:, 1]
plt.figure(figsize=(10, 5))
plt.plot(iterations, train_loss)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Training curve")
plt.show()

### Test network predictions

In [ ]:
TEST_BATCH_SIZE = 128

model.test(test_set_flat, test_labels, TEST_BATCH_SIZE)

In [ ]:
example = np.random.choice(np.arange(n_test))

sample = np.expand_dims(test_set_flat[example], axis=0)
label = np.expand_dims(test_labels[example], axis=0)

digit = np.where(label[0]==1.0)[0][0]

feed_dict = {model.input: sample, model.ground_truth: label}
prediction = model.sess.run(model.prediction, feed_dict=feed_dict)[0]

image = np.reshape(sample, (32, 32))

print("Test sample digit: {}".format(digit))
fig, ax = plt.subplots(1, 2, figsize=(17, 5))
ax[0].imshow(image, cmap='gray')
ax[0].set_title("Test example")

classes = np.arange(10)
width = 1.0

#fig, ax = plt.subplots()
ax[1].bar(classes, prediction, width, color='Blue')
ax[1].set_ylabel('Probabilities')
ax[1].set_title('Network categorical distribution')
ax[1].set_xticks(classes)
ax[1].set_xticklabels(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))
ax[1].set_xlabel('Digit class')

plt.show()

print("Network prediction probabilities:")
print(prediction)

In [ ]:
model.sess.close()

## Exercise: experiment with different network hyperparameters to try to improve the performance of the model.